<a href="https://colab.research.google.com/github/mirkhalilrehman/Pytorch-Lightning/blob/main/Pytorch_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install pytorch_lightning

In [15]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchmetrics import Accuracy, F1Score, Precision
from torchmetrics import Metric


In [16]:
class MNistDataModule(pl.LightningDataModule):
    def __init__(self, data_dir, batch_size, num_workers):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers

    def prepare_data(self):
        datasets.MNIST(self.data_dir, train=True, download=True)
        datasets.MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        entire_dataset = datasets.MNIST(
            root=self.data_dir, train=True, transform=transforms.ToTensor(), download=False
        )
        self.train_ds, self.val_ds = random_split(entire_dataset, [50000, 10000])
        self.test_ds = datasets.MNIST(
            root=self.data_dir, train=False, transform=transforms.ToTensor(), download=False
        )

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=False)

In [18]:
from torchmetrics import Accuracy, F1Score, Precision

class NN(pl.LightningModule):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        self.loss_f = nn.CrossEntropyLoss()

        self.accuracy = Accuracy(task="multiclass", num_classes=num_classes)
        self.f1_score = F1Score(task="multiclass", num_classes=num_classes)
        self.precision = Precision(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def common_step(self, batch, batch_idx):
        x, y = batch
        x = x.reshape(x.size(0), -1)
        scores = self.forward(x)
        loss = self.loss_f(scores, y)
        return loss, scores, y

    def training_step(self, batch, batch_idx):
        loss, scores, y = self.common_step(batch, batch_idx)

        # Log metrics
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_accuracy", self.accuracy(scores, y), on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_f1_score", self.f1_score(scores, y), on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_precision", self.precision(scores, y), on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, scores, y = self.common_step(batch, batch_idx)

        # Log metrics
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_accuracy", self.accuracy(scores, y), on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_f1_score", self.f1_score(scores, y), on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_precision", self.precision(scores, y), on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss, scores, y = self.common_step(batch, batch_idx)

        # Log metrics
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_accuracy", self.accuracy(scores, y), on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_f1_score", self.f1_score(scores, y), on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_precision", self.precision(scores, y), on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)


In [19]:
input_size = 784
num_classes = 10
batch_size = 64

In [20]:
model = NN(input_size=input_size, num_classes=num_classes)

dm = MNistDataModule(data_dir="dataset/",batch_size=batch_size,num_workers=4)
trainer=pl.Trainer(accelerator="gpu",devices=1,min_epochs=1,max_epochs=6,precision=16)
trainer.fit(model,dm)
trainer.validate(model,dm)
trainer.test(model, dm)

/usr/local/lib/python3.11/dist-packages/lightning_fabric/connector.py:572: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 17.6MB/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 483kB/s]


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       val_accuracy        │    0.9732999801635742     │
│       val_f1_score        │    0.9732999801635742     │
│         val_loss          │    0.0963115319609642     │
│       val_precision       │    0.9732999801635742     │
└───────────────────────────┴───────────────────────────┘

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.963100016117096     │
│       test_f1_score       │     0.963100016117096     │
│         test_loss         │    0.11883746087551117    │
│      test_precision       │     0.963100016117096     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.11883746087551117,
  'test_accuracy': 0.963100016117096,
  'test_f1_score': 0.963100016117096,
  'test_precision': 0.963100016117096}]